In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Hemochromatosis"
cohort = "GSE50579"

# Input paths
in_trait_dir = "../../input/GEO/Hemochromatosis"
in_cohort_dir = "../../input/GEO/Hemochromatosis/GSE50579"

# Output paths
out_data_file = "../../output/preprocess/Hemochromatosis/GSE50579.csv"
out_gene_data_file = "../../output/preprocess/Hemochromatosis/gene_data/GSE50579.csv"
out_clinical_data_file = "../../output/preprocess/Hemochromatosis/clinical_data/GSE50579.csv"
json_path = "../../output/preprocess/Hemochromatosis/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait (Hemochromatosis) appears in row 1 under "etiology: genetic hemochromatosis"
trait_row = 1
# Age appears in row 5
age_row = 5
# Gender appears in row 3
gender_row = 3

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait values to binary (0 or 1)."""
    if value is None or 'n.d.' in value:
        return None
    # Extract the value after ':'
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # If the value indicates genetic hemochromatosis, return 1, otherwise return 0
    if 'genetic hemochromatosis' in value.lower():
        return 1
    return 0

def convert_age(value):
    """Convert age values to continuous numeric values."""
    if value is None or 'n.d.' in value:
        return None
    # Extract the value after ':'
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        # Convert to integer
        return int(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender values to binary (0 for female, 1 for male)."""
    if value is None or 'n.d.' in value:
        return None
    # Extract the value after ':'
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'female' in value.lower():
        return 0
    elif 'male' in value.lower():
        return 1
    return None

# 3. Save Metadata
# Determine trait availability
is_trait_available = trait_row is not None
# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# We're skipping this step for now as the required clinical_data.csv file 
# is not yet available in the input directory.
# This will be handled in a subsequent step when the clinical data is ready.


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll correct the code for the current step to properly explore available files and handle the clinical data.

```python
import pandas as pd
import json
import os
from typing import Callable, Optional, Dict, Any
import numpy as np
import glob

# Let's first explore the directory to find what files are available
print(f"Exploring directory: {in_cohort_dir}")
available_files = glob.glob(os.path.join(in_cohort_dir, "*"))
print(f"Available files: {available_files}")

# Look for series matrix files which typically contain clinical data in GEO datasets
series_matrix_files = glob.glob(os.path.join(in_cohort_dir, "*series_matrix*"))
print(f"Series matrix files: {series_matrix_files}")

# Check for any other files that might contain clinical data
clinical_files = [f for f in available_files if 'clinical' in f.lower() or 'sample' in f.lower()]
print(f"Potential clinical files: {clinical_files}")

# Initialize variables
clinical_data = None
sample_characteristics = {}
is_gene_available = False
is_trait_available = False
trait_row = None
age_row = None
gender_row = None

# Try to load clinical data from various potential sources
if series_matrix_files:
    try:
        # Try to parse series matrix file which often contains clinical data
        with open(series_matrix_files[0], 'r') as file:
            lines = file.readlines()
            
            # Extract sample characteristics sections from series matrix
            for i, line in enumerate(lines):
                if line.startswith('!Sample_title') or line.startswith('!Sample_geo_accession'):
                    # Get sample identifiers
                    parts = line.strip().split('\t')
                    sample_ids = parts[1:]
                
                if line.startswith('!Sample_characteristics_ch1'):
                    # Get sample characteristics
                    parts = line.strip().split('\t')
                    values = parts[1:]
                    
                    # Store in sample_characteristics with index as key
                    if i not in sample_characteristics:
                        sample_characteristics[i] = values
            
            # Check if the file contains gene expression data
            for line in lines:
                if '!Series_platform_id' in line or '!Platform_title' in line:
                    platform = line.lower()
                    # Most common gene expression platforms
                    if ('illumina' in platform or 'affymetrix' in platform or 
                        'agilent' in platform or 'gene expression' in platform or 
                        'transcriptome' in platform or 'rna' in platform):
                        is_gene_available = True
                        break
    except Exception as e:
        print(f"Error parsing series matrix file: {e}")

# If we couldn't load clinical data from series matrix, try other approaches
if not sample_characteristics and clinical_files:
    try:
        # Try to read other clinical files
        for file_path in clinical_files:
            if file_path.endswith('.csv'):
                df = pd.read_csv(file_path)
            elif file_path.endswith('.txt') or file_path.endswith('.tsv'):
                df = pd.read_csv(file_path, sep='\t')
            
            # Check if this file contains relevant clinical data
            if 'characteristic' in ' '.join(df.columns).lower():
                clinical_data = df
                break
    except Exception as e:
        print(f"Error loading alternative clinical files: {e}")

# If we still don't have clinical data, make a best-guess about gene availability
if not is_gene_available:
    # If there are any files that might contain gene expression data
    gene_expr_files = [f for f in available_files if 'expr' in f.lower() or 'gene' in f.lower()]
    is_gene_available = len(gene_expr_files) > 0

# Print sample characteristics for analysis
print("Sample Characteristics:")
for key, value in sample_characteristics.items():
    if isinstance(value, list) and len(value) > 0:
        print(f"{key}: {value[:5]}...")  # Show first 5 values
    else:
        print(f"{key}: {value}")

# Look for keys that might contain trait, age, and gender information
for key, values in sample_characteristics.items():
    if isinstance(values, list) and len(values) > 0:
        unique_values = set(values)
        values_str = str(values)
        
        # Check for trait-related information (Hemochromatosis)
        if any(term in values_str.lower() for term in ["hemochromatosis", "iron", "hfe", "patient", "disease"]):
            if len(unique_values) > 1:  # Ensure it's not a constant feature
                trait_row = key
                is_trait_available = True
        
        # Check for age information
        if "age" in values_str.lower():
            if len(unique_values) > 1:  # Ensure it's not a constant feature
                age_row = key
        
        # Check for gender/sex information
        if any(term in values_str.lower() for term in ["gender", "sex", "male", "female"]):
            if len(unique_values) > 1:  # Ensure it's not a constant feature
                gender_row = key

# Define conversion functions
def convert_trait(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value_lower = str(value).lower()
    
    # Identify cases
    if any(term in value_lower for term in ["hemochromatosis", "hfe", "patient", "disease", "case"]):
        return 1
    # Identify controls
    elif any(term in value_lower for term in ["control", "normal", "healthy"]):
        return 0
    else:
        return None

def convert_age(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract age as a number
    value_str = str(value).lower()
    try:
        # Extract numbers from the string
        import re
        numbers = re.findall(r'\d+', value_str)
        if numbers:
            age = int(numbers[0])
            return age
        else:
            return None
    except:
        return None

def convert_gender(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    value_lower = str(value).lower()
    
    if any(term == value_lower for term in ["female", "f"]):
        return 0
    elif any(term == value_lower for term in ["male", "m"]):
        return 1
    else:
        return None

# Save metadata - initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# If clinical data is available, extract and save clinical features
if is_trait_available and trait_row is not None and sample_characteristics:
    # Create a DataFrame from sample_characteristics for geo_select_clinical_features
    sample_df = pd.DataFrame()
    for key, values in sample_characteristics.items():
        if isinstance(values, list):
            sample_df[key] = values
    
    # Extract clinical features using the library function
    try:
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=sample_df,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row


### Step 4: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
import gzip
from typing import Optional, Callable, Dict, Any

# For GSE50579, the clinical data is embedded in the series matrix file
series_matrix_path = os.path.join(in_cohort_dir, "GSE50579_series_matrix.txt.gz")

# Extract clinical data from series matrix file
clinical_data = pd.DataFrame()
sample_chars = []
sample_ids = []

try:
    with gzip.open(series_matrix_path, 'rt') as f:
        # Process the file line by line to extract sample characteristics
        for line in f:
            if line.startswith('!Sample_geo_accession'):
                # Extract sample IDs
                sample_ids = line.strip().split('\t')[1:]
            elif line.startswith('!Sample_characteristics_ch1'):
                # Extract sample characteristics
                chars = line.strip().split('\t')[1:]
                sample_chars.append(chars)
            elif line.startswith('!Sample_title'):
                # Extract sample titles for possible trait information
                sample_titles = line.strip().split('\t')[1:]
            # Once we've passed the header section, we can stop
            elif line.startswith('!series_matrix_table_begin'):
                break
    
    # Create a DataFrame from the extracted characteristics
    if sample_chars and sample_ids:
        clinical_data = pd.DataFrame(sample_chars, columns=sample_ids)
        # Transpose so that each row represents a characteristic and each column a sample
        clinical_data = clinical_data.T
        
        print("Clinical data shape:", clinical_data.shape)
        print("Clinical data preview:")
        print(clinical_data.head())
        
        # Check unique values for each row to identify relevant information
        for i in range(len(clinical_data.columns)):
            unique_values = clinical_data.iloc[:, i].unique()
            print(f"Row {i}: {len(unique_values)} unique values")
            print(f"Sample values: {unique_values[:5] if len(unique_values) > 5 else unique_values}")
    
except Exception as e:
    print(f"Error loading clinical data from series matrix: {e}")
    clinical_data = pd.DataFrame()

# Confirm gene expression data is available from series matrix file
is_gene_available = os.path.exists(series_matrix_path)
print(f"Gene expression data available: {is_gene_available}")

# Based on examination of the data, set trait_row, age_row, gender_row
# These values would be set based on the actual data review
trait_row = None
age_row = None
gender_row = None

# After reviewing the unique values in each row, identify the appropriate rows
# This is sample code - in an actual implementation, these would be determined after examining the data
if not clinical_data.empty:
    # Look for disease status in sample characteristics
    for i in range(len(clinical_data.columns)):
        col_values = [str(val).lower() for val in clinical_data.iloc[:, i].unique()]
        col_values_str = ' '.join(col_values)
        
        # Look for trait-related information (hemochromatosis)
        if any(term in col_values_str for term in ['hemochromatosis', 'iron overload', 'hfe', 'disease', 'status', 'diagnosis', 'condition']):
            trait_row = i
            print(f"Found potential trait row: {i}")
            print(f"Values: {clinical_data.iloc[:, i].unique()}")
        
        # Look for age information
        elif any(term in col_values_str for term in ['age', 'years']):
            age_row = i
            print(f"Found potential age row: {i}")
            print(f"Values: {clinical_data.iloc[:, i].unique()}")
        
        # Look for gender/sex information
        elif any(term in col_values_str for term in ['gender', 'sex', 'male', 'female']):
            gender_row = i
            print(f"Found potential gender row: {i}")
            print(f"Values: {clinical_data.iloc[:, i].unique()}")

# Define conversion functions with proper value extraction
def extract_value(value_str):
    if isinstance(value_str, str) and ':' in value_str:
        return value_str.split(':', 1)[1].strip()
    return value_str

def convert_trait(value):
    if not isinstance(value, str):
        return None
    
    value = extract_value(value).lower()
    # Convert hemochromatosis-related terms to binary
    if any(term in value for term in ['control', 'normal', 'healthy', 'wild type', 'wt']):
        return 0
    elif any(term in value for term in ['hemochromatosis', 'iron overload', 'hfe', 'patient', 'homozygous', 'c282y']):
        return 1
    else:
        return None

def convert_age(value):
    if not isinstance(value, str):
        return None
        
    value = extract_value(value)
    try:
        # Extract numeric value from string like "age: 45 years"
        import re
        match = re.search(r'(\d+)', value)
        if match:
            return float(match.group(1))
        return float(value)
    except:
        return None

def convert_gender(value):
    if not isinstance(value, str):
        return None
        
    value = extract_value(value).lower()
    if any(term in value for term in ['female', 'f']):
        return 0
    elif any(term in value for term in ['male', 'm']):
        return 1
    else:
        return None

# Determine trait availability
is_trait_available = trait_row is not None

# Save metadata about the dataset
initial_validation = validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Extract clinical features if trait_row is not None
if is_trait_available:
    try:
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted features
        preview = preview_df(clinical_features)
        print("\nClinical Features Preview:")
        for key, values in preview.items():
            print(f"{key}: {values}")
        
        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except Exception as e:
        print(f"Error extracting clinical features: {e}")
        is_trait_available = False
        # Re-save metadata
        validate_and_save_cohort_info(
            is_final=False,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available
        )
else:
    print("No trait data available, skipping clinical feature extraction")


### Step 5: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


### Step 6: Gene Identifier Review

In [ ]:
# Reviewing gene identifiers
# Based on the identifiers displayed (like "(+)E1A_r60_1", "A_19_P00315452"), these appear to be 
# microarray probe IDs rather than standard human gene symbols.
# Standard human gene symbols would look like "BRCA1", "TP53", "HFE", etc.
# These probe IDs will need to be mapped to actual gene symbols.

requires_gene_mapping = True


### Step 7: Gene Annotation

In [ ]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


### Step 8: Gene Identifier Mapping

In [ ]:
# 1. Identify the appropriate columns for gene mapping
# Based on the gene annotation preview, 'ID' contains the probe identifiers
# and 'GENE_SYMBOL' likely contains the corresponding gene symbols
probe_id_col = 'ID'
gene_symbol_col = 'GENE_SYMBOL'

print(f"Using columns for mapping: Probe ID = '{probe_id_col}', Gene Symbol = '{gene_symbol_col}'")

# 2. Create a gene mapping dataframe by extracting these two columns
try:
    # Extract the mapping between probe IDs and gene symbols
    mapping_df = get_gene_mapping(gene_annotation, probe_id_col, gene_symbol_col)
    print(f"Created mapping with {len(mapping_df)} rows")
    print("Sample of mapping data:")
    print(mapping_df.head())
    
    # Check for any issues with the mapping data
    num_mapped_probes = len(mapping_df['ID'].unique())
    num_mapped_genes = len(mapping_df['Gene'].unique())
    print(f"Number of unique probes with gene mapping: {num_mapped_probes}")
    print(f"Number of unique genes in mapping: {num_mapped_genes}")
    
    # 3. Apply the gene mapping to convert probe-level expression to gene-level expression
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    print(f"Converted probe-level data to gene-level data")
    print(f"Number of genes in final expression matrix: {len(gene_data)}")
    
    # Save the gene expression data to CSV
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error during gene mapping: {e}")
    # If mapping fails, attempt to extract using other common gene-related columns
    try:
        print("Attempting alternative mapping approach...")
        # Some platforms use different column names for gene symbols
        alternative_gene_cols = ['GENE_SYMBOL', 'GENE', 'SYMBOL', 'GeneSymbol', 'Gene_Symbol', 'GENE NAME', 'GENE_NAME']
        
        for gene_col in alternative_gene_cols:
            if gene_col in gene_annotation.columns:
                print(f"Trying alternative gene symbol column: {gene_col}")
                mapping_df = get_gene_mapping(gene_annotation, probe_id_col, gene_col)
                if not mapping_df.empty and mapping_df['Gene'].notna().any():
                    gene_data = apply_gene_mapping(gene_data, mapping_df)
                    print(f"Alternative mapping successful with column {gene_col}")
                    print(f"Number of genes in final expression matrix: {len(gene_data)}")
                    break
        
        # If we successfully mapped with an alternative column, save the data
        if 'gene_data' in locals() and not gene_data.empty:
            os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
            gene_data.to_csv(out_gene_data_file)
            print(f"Gene expression data saved to {out_gene_data_file}")
    except Exception as e2:
        print(f"Alternative mapping also failed: {e2}")
        print("Unable to map probes to gene symbols")


### Step 9: Data Normalization and Linking

In [ ]:
import numpy as np

# Define the conversion functions that were defined in a previous step
def convert_trait(value):
    """Convert trait values to binary (0 or 1)."""
    if value is None or 'n.d.' in value:
        return None
    # Extract the value after ':'
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # If the value indicates genetic hemochromatosis, return 1, otherwise return 0
    if 'genetic hemochromatosis' in value.lower():
        return 1
    return 0

def convert_age(value):
    """Convert age values to continuous numeric values."""
    if value is None or 'n.d.' in value:
        return None
    # Extract the value after ':'
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        # Convert to integer
        return int(value)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender values to binary (0 for female, 1 for male)."""
    if value is None or 'n.d.' in value:
        return None
    # Extract the value after ':'
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'female' in value.lower():
        return 0
    elif 'male' in value.lower():
        return 1
    return None

# 1. Normalize gene symbols in the gene expression data
try:
    # Load the gene expression data that was saved in a previous step
    gene_data = pd.read_csv(out_gene_data_file, index_col=0)
    print(f"Loaded gene data shape before normalization: {gene_data.shape}")
    
    # Check if gene data is empty
    if gene_data.shape[0] == 0:
        print("Warning: Gene data is empty. Attempting to load from matrix file directly.")
        # Extract gene expression data directly from matrix file as a fallback
        gene_data = get_genetic_data(matrix_file)
    
    # Normalize gene symbols using the NCBI Gene database information
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Check if normalization retained any genes
    if normalized_gene_data.shape[0] == 0:
        print("Warning: Gene symbol normalization resulted in zero genes. Using original gene data instead.")
        normalized_gene_data = gene_data
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Gene data saved to {out_gene_data_file}")
    
    is_gene_available = normalized_gene_data.shape[0] > 0
    print(f"Gene data available: {is_gene_available}")
except Exception as e:
    print(f"Error loading or normalizing gene data: {e}")
    is_gene_available = False

# 2. Load the clinical data
try:
    # First we need to get the clinical data from the original source
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_info, clinical_df = get_background_and_clinical_data(matrix_file)
    
    # Create clinical features dataframe
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_df,
        trait=trait,
        trait_row=1,  # etiology info is in row 1
        convert_trait=convert_trait,
        age_row=5,    # age info is in row 5
        convert_age=convert_age,
        gender_row=3, # gender info is in row 3
        convert_gender=convert_gender
    )
    
    # Check if we have trait data
    is_trait_available = trait in clinical_features.index
    print(f"Trait '{trait}' available: {is_trait_available}")
    
    # Save the properly extracted clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")
    
except Exception as e:
    print(f"Error processing clinical data: {e}")
    clinical_features = pd.DataFrame()
    is_trait_available = False

# 3. Link clinical and genetic data
if is_gene_available and is_trait_available:
    try:
        # Link clinical features with gene expression data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        
        # 4. Handle missing values systematically
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # 5. Check if trait and demographic features are biased
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
        print(f"Is dataset biased for {trait}: {is_biased}")
        
        # Additional note about the dataset
        note = f"Dataset contains gene expression data with {linked_data.shape[0]} samples and {linked_data.shape[1] - (3 if 'Age' in linked_data.columns and 'Gender' in linked_data.columns else 2 if 'Age' in linked_data.columns or 'Gender' in linked_data.columns else 1)} genes."
        
    except Exception as e:
        print(f"Error processing linked data: {e}")
        is_biased = True
        linked_data = pd.DataFrame({trait: [np.nan]})
        note = f"Error during data processing: {e}"
else:
    # If either gene or trait data is unavailable, set up minimal data for validation
    is_biased = True
    if is_gene_available:
        note = f"Dataset contains gene expression data but no {trait} measurements. Not usable for studying {trait} associations."
    elif is_trait_available:
        note = f"Dataset contains {trait} data but no gene expression data. Not usable for genetic associations."
    else:
        note = f"Dataset contains neither {trait} data nor gene expression data. Not usable for association studies."
    
    # Create a minimal dataframe for validation
    linked_data = pd.DataFrame({trait: [np.nan]})

# 6. Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased if 'is_biased' in locals() else True,
    df=linked_data if 'linked_data' in locals() else pd.DataFrame({trait: [np.nan]}),
    note=note if 'note' in locals() else "Error during data processing"
)

# Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Linked data not saved.")


### Step 10: Data Normalization and Linking

In [ ]:
import numpy as np
import os
import gzip

# 1. Extract gene expression data using the alternative approach that worked in Step 3
with gzip.open(matrix_file, 'rt') as file:
    # Find the start of the data
    for line in file:
        if '!series_matrix_table_begin' in line:
            break
            
    # Read the headers and data
    gene_data = pd.read_csv(file, sep='\t', index_col=0)

# Check if we have gene data before proceeding
if gene_data.empty:
    print("No gene expression data found in the matrix file.")
    is_gene_available = False
else:
    is_gene_available = True
    print(f"Gene data shape before normalization: {gene_data.shape}")

    # Normalize gene symbols using the NCBI Gene database information
    try:
        normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
        
        # Save the normalized gene data to the output file
        os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
        normalized_gene_data.to_csv(out_gene_data_file)
        print(f"Normalized gene data saved to {out_gene_data_file}")
    except Exception as e:
        print(f"Error normalizing gene data: {e}")
        is_gene_available = False
        normalized_gene_data = gene_data  # Use original data if normalization fails

# 2. Link clinical and genetic data
# In Step 2, we determined that Height data is not available in this dataset (trait_row = None)
# Create a minimal clinical dataframe with the trait column (containing NaNs)
if is_gene_available:
    sample_ids = gene_data.columns
    minimal_clinical_df = pd.DataFrame(index=sample_ids)
    minimal_clinical_df[trait] = np.nan  # Add the trait column with NaN values

    # If we have age and gender data from Step 2, add those columns
    if age_row is not None:
        minimal_clinical_df['Age'] = get_feature_data(clinical_data, age_row, 'Age', convert_age).iloc[0]

    if gender_row is not None:
        minimal_clinical_df['Gender'] = get_feature_data(clinical_data, gender_row, 'Gender', convert_gender).iloc[0]

    minimal_clinical_df.index.name = 'Sample'

    # Save this minimal clinical data for reference
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    minimal_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")

    # Create a linked dataset 
    if is_gene_available and normalized_gene_data is not None:
        linked_data = pd.concat([minimal_clinical_df, normalized_gene_data.T], axis=1)
        linked_data.index.name = 'Sample'
        print(f"Linked data shape: {linked_data.shape}")
    else:
        linked_data = minimal_clinical_df
        print("No gene data to link with clinical data.")
else:
    # Create a minimal dataframe with just the trait for the validation step
    linked_data = pd.DataFrame({trait: [np.nan]})
    print("No gene data available, creating minimal dataframe for validation.")

# 4 & 5. Validate and save cohort information
# Since trait_row was None in Step 2, we know Height data is not available
is_trait_available = False  # Height data is not available

note = "Dataset contains gene expression data but no Height measurements. This dataset is not usable for studying Height associations."

# For datasets without trait data, we set is_biased to False
# This indicates the dataset is not usable due to missing trait data, not due to bias
is_biased = False

# Final validation
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Since there is no trait data, the dataset is not usable for our association study
# So we should not save it to out_data_file
print(f"Dataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset does not contain Height data and cannot be used for association studies.")